# 1. Setup

In [1]:
#Update the date with day and day+1. For example, if we want to run monitoring of 09/01/2022, then st = '2022-09-01' and ed = '2022-09-02' 
st = '2022-11-01'
ed = '2022-11-02' 

In [2]:
from kplib.dumbo import Dumbo, FeatureOptions, EvaluationOptions, DataframeInput, FileInput
import pandas as pd
import numpy as np
from pysnowflake.simple import Session
import blocks

/usr/local/lib/python3.7/dist-packages/pysnowflake/__init__.py:21: FutureWarning: 
    With pysnowflake version 1.0.0 there will be a breaking change.
    The legacy api will be dropped as it is no longer maintained, and simple will become the only api.
    This will make it easier to keep support for newer features like AWS support, etc. The expected change is
    the start of Q2 2022.
    
  FutureWarning,


In [3]:
def PullToken(st_dt, ed_dt):
    with Session() as ss:
        df  = ss.download(query="""
            SELECT GET(entity_tokens, 'unit_token') as unit_token
            , decision_event_token as decision_event_token
            , created_at as created_at
            , status
            FROM decisions.public.decisions_data_api dda
            -- Filter by Date
            WHERE created_at >= '{}' ::DATE
            AND created_at < '{}' ::DATE
            --   Get Production decisions only
            AND is_shadow=FALSE
            AND decision_type = 'ELIGIBILITY';
            """.format(st_dt, ed_dt)) 
    return df

In [4]:
def PullSignal(feature_list):
    event_df = Dumbo().get_features(
        "CAPITAL_DECISION",
        # pass df and specify token
        DataframeInput(
            df, token_col="decision_event_token"
        ),
        # pass feature_list here
        # include_for_rules_only: some signals are only for rules
        feature_options=FeatureOptions(features=feature_list,
                                      include_for_rules_only=True),
        # need to pass acl so we have permissions to query
        evaluation_options=EvaluationOptions(acl_reasons=["capital_risk_assessment"],
                                             # remove examples without data (happens if Dumbo is too stale)
                                            remove_unknown_results=True
                                            )
    )
    event_df.drop(columns=['timestamp'], inplace = True)
    
    return event_df

In [5]:
def PullRule5(st_dt, ed_dt):
    with Session() as ss:
        df  = ss.download(query="""
            select
            os.*
            from
            app_capital.app_capital.offer_sets os
            where
            os.product_id = 3
            and os.offer_created_at between '{}' and '{}'
            and os.COUNTRY_CODE = 'US'
            and os.MERCHANT_BUSINESS_TYPE in ('charitible_orgs','charitable_social_service_organizations','political_organizations','tourism','travel_tourism','travel_agencies_and_tour_operators');
            """.format(st_dt, ed_dt)) 
    return df

In [6]:
feature_list1 = ["millis_since_last_capture_by_merchant_from_capital_decision_merchant"]
feature_list3 = ["total_net_amount_in_full_28d_by_merchant_from_capital_decision_merchant", "total_success_amount_in_full_7d_by_merchant_from_capital_decision_merchant"]
feature_list4 = ["millis_since_first_success_auth_by_merchant_from_capital_decision_merchant"]
feature_list6 = ["total_chargeback_v3_count_in_full_28d_by_merchant_from_capital_decision_merchant"]
feature_list7 = ["total_net_amount_in_full_365d_by_merchant_from_capital_decision_merchant", "scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant", "total_net_amount_in_full_28d_by_merchant_from_capital_decision_merchant"]
feature_list8 = ["total_net_count_in_full_365d_by_merchant_from_capital_decision_merchant", "scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant", "total_net_count_in_full_28d_by_merchant_from_capital_decision_merchant"]
feature_list9 = ["total_active_day_count_in_full_28d_by_merchant_from_capital_decision_merchant", "total_active_day_count_in_full_365d_by_merchant_from_capital_decision_merchant", "scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant"]
feature_list10 = ["active_payment_limit_count_by_merchant_from_capital_decision_merchant"]

In [7]:
thresholds_data = {'Rule': ['1', '3, 7', '3', '4', '6', '7', '7, 8, 9', '8', '8', '9', '9', '10'],
                   'Signal':["millis_since_last_capture_by_merchant_from_capital_decision_merchant", 
                             "total_net_amount_in_full_28d_by_merchant_from_capital_decision_merchant", 
                             "total_success_amount_in_full_7d_by_merchant_from_capital_decision_merchant",
                             "millis_since_first_success_auth_by_merchant_from_capital_decision_merchant", 
                             "total_chargeback_v3_count_in_full_28d_by_merchant_from_capital_decision_merchant", 
                             "total_net_amount_in_full_365d_by_merchant_from_capital_decision_merchant", 
                             "scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant", 
                             "total_net_count_in_full_365d_by_merchant_from_capital_decision_merchant", 
                             "total_net_count_in_full_28d_by_merchant_from_capital_decision_merchant", 
                             "total_active_day_count_in_full_28d_by_merchant_from_capital_decision_merchant", 
                             "total_active_day_count_in_full_365d_by_merchant_from_capital_decision_merchant", 
                             "active_payment_limit_count_by_merchant_from_capital_decision_merchant"],
                  'Lower':[4150550, 109808, 22900, 44722474707, 0, 1255656, 1, 114, 10, 7, 65, 0],
                  'Upper':[72488589, 712401, 185358, 198443965759, 0, 8213163, 1, 898, 76, 19, 220, 0]}

median_thresholds = pd.DataFrame(thresholds_data)

# 2. Pull Token

In [8]:
df = PullToken(st, ed) 

In [9]:
df #check if the date is correct

,unit_token,decision_event_token,created_at,status
0,L0KFYAAR102K8,DS-CS_C-GE8ZPNM1W50CXM_2022-11-01,2022-11-01 19:00:46.606,declined
1,L21GZC4A1MQ9M,DS-CS_C-1EZF45M315QQT9_2022-11-01,2022-11-01 19:28:43.250,declined
2,LN2NP1WKH3JC4,DS-CS_C-2HTZAV64EG04AN_2022-11-01,2022-11-01 18:59:48.712,declined
3,LMNJQZ2ATE4J2,DS-CS_C-121XFZ41YS72RF_2022-11-01,2022-11-01 19:16:46.108,declined
4,E8B09ETGJP3PR,DS-CS_C-1E97FB9QYHCQXF_2022-11-01,2022-11-01 19:21:23.100,declined
...,...,...,...,...
1648234,PFGZEZDE78KQS,DS-CS_C-2SRSMW6RH28WX2_2022-11-01,2022-11-01 15:22:39.136,approved
1648235,K8FBQ4FY0QNZV,DS-CS_C-1KJA212XPM0J0E_2022-11-01,2022-11-01 15:25:16.103,approved
1648236,DZRD8GC5J9Z8P,DS-CS_C-19F8108YT4XFAJ_2022-11-01,2022-11-01 15:28:55.099,declined
1648237,FKQVTZY2EQ3CY,DS-CS_C-1VQ4B0ZF1EXXX6_2022-11-01,2022-11-01 15:23:18.710,approved


# 3. Pull Signals

In [10]:
feature_list = list(set(feature_list1 + feature_list3 + feature_list4 + feature_list6 + feature_list7 + feature_list8 + feature_list9 + feature_list10))
event_df = PullSignal(feature_list)

'Waiting for job 234635 to launch'

'Job 234635 is running at https://console.cloud.google.com/dataflow/jobs/us-central1/2022-11-11_18_13_49-7675689761341846807?project=risk-144219'

'Job 234635 finished successfully'

'Checking output for errors ...'

'No errors found in output'

In [11]:
event_df

,unit_token,decision_event_token,created_at,status,total_chargeback_v3_count_in_full_28d_by_merchant_from_capital_decision_merchant,total_success_amount_in_full_7d_by_merchant_from_capital_decision_merchant,total_net_amount_in_full_28d_by_merchant_from_capital_decision_merchant,total_net_count_in_full_365d_by_merchant_from_capital_decision_merchant,total_net_count_in_full_28d_by_merchant_from_capital_decision_merchant,millis_since_first_success_auth_by_merchant_from_capital_decision_merchant,total_active_day_count_in_full_28d_by_merchant_from_capital_decision_merchant,total_active_day_count_in_full_365d_by_merchant_from_capital_decision_merchant,millis_since_last_capture_by_merchant_from_capital_decision_merchant,active_payment_limit_count_by_merchant_from_capital_decision_merchant,total_net_amount_in_full_365d_by_merchant_from_capital_decision_merchant,scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant
0,C98A6PXQNVKJX,DS-CS_C-3QD8QVNCNGMM4Z_2022-11-01,2022-11-01 15:37:03.554000+00:00,approved,0.0,789300.0,2586300.0,2610.0,207.0,1.686222e+11,27.0,328.0,53097894.0,0.0,34982309.0,1.000000
1,L1AWM1BH46N4J,DS-CS_C-MG5ZZAQPJDT131_2022-11-01,2022-11-01 15:25:27.229000+00:00,approved,0.0,44078.0,170940.0,70.0,10.0,1.572108e+10,7.0,42.0,8843028.0,0.0,1054062.0,2.005969
2,683B5SK26RMNW,DS-CS_C-28RBF8CNGFDB4N_2022-11-01,2022-11-01 18:26:23.362000+00:00,declined,0.0,136730.0,209276.0,26.0,5.0,2.514948e+11,5.0,20.0,95158212.0,0.0,973156.0,1.000000
3,95EJV7HCQN7AY,DS-CS_C-100NWRKKK11DKV_2022-11-01,2022-11-01 18:24:29.134000+00:00,approved,0.0,79900.0,365600.0,312.0,22.0,9.770669e+10,13.0,181.0,72174862.0,0.0,6898025.0,1.000000
4,LQFX8XVZC8EF0,DS-CS_C-21CEEAABVYM9SB_2022-11-01,2022-11-01 18:08:31.883000+00:00,declined,0.0,189634.0,1139185.0,335.0,85.0,1.372977e+10,18.0,91.0,3355993.0,0.0,4870907.0,2.296907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8236,L5VMH4X0QAPFJ,DS-CS_C-3BEF9S9H2W6KDZ_2022-11-01,2022-11-01 18:11:23.106000+00:00,declined,0.0,8000.0,92500.0,450.0,24.0,1.483260e+10,6.0,43.0,398502377.0,0.0,7736656.0,2.126127
8237,L1MXM20WQEKH5,DS-CS_C-P70BN5KVETZQZJ_2022-11-01,2022-11-01 15:01:38.300000+00:00,declined,0.0,93635.0,300805.0,454.0,46.0,5.737327e+10,15.0,176.0,338086498.0,0.0,2807305.0,1.000000
8238,7WB20BGZRMPW4,DS-CS_C-NEE76DT0KTRFPM_2022-11-01,2022-11-01 15:40:01.822000+00:00,declined,0.0,152000.0,152000.0,51.0,4.0,2.125188e+11,4.0,36.0,321521750.0,0.0,1468866.0,1.000000
8239,BX3X5CPS9R6T9,DS-CS_C-NHE3YH5DJB43V2_2022-11-01,2022-11-01 15:21:57.110000+00:00,declined,0.0,382556.0,3603000.0,229.0,66.0,2.302515e+11,19.0,66.0,253263714.0,0.0,13992424.0,1.000000


# 4. Calculate Medians and Compare to Thresholds

In [12]:
median_value = event_df[feature_list].median()
df_median = pd.DataFrame({'Signal':median_value.index, 'Median':median_value.values})
df_compare = median_thresholds.merge(df_median, how='left', on='Signal')

conditions = [
    (df_compare['Median'] < df_compare['Lower']) | ( df_compare['Median'] > df_compare['Upper']),
    (df_compare['Median'] >= df_compare['Lower']) & ( df_compare['Median'] <= df_compare['Upper'])
]
values = ['Alert', 'Normal']
df_compare['Result'] = np.select(conditions, values)
format_dict = {'Median': '{:.0f}'}
df_compare.style.format(format_dict)

,Rule,Signal,Lower,Upper,Median,Result
0,1,millis_since_last_capture_by_merchant_from_capital_decision_merchant,4150550,72488589,134855755,Alert
1,"3, 7",total_net_amount_in_full_28d_by_merchant_from_capital_decision_merchant,109808,712401,256300,Normal
2,3,total_success_amount_in_full_7d_by_merchant_from_capital_decision_merchant,22900,185358,62357,Normal
3,4,millis_since_first_success_auth_by_merchant_from_capital_decision_merchant,44722474707,198443965759,102577131920,Normal
4,6,total_chargeback_v3_count_in_full_28d_by_merchant_from_capital_decision_merchant,0,0,0,Normal
5,7,total_net_amount_in_full_365d_by_merchant_from_capital_decision_merchant,1255656,8213163,2668377,Normal
6,"7, 8, 9",scale_factor_time_since_first_success_card_auth_by_merchant_from_capital_decision_merchant,1,1,1,Normal
7,8,total_net_count_in_full_365d_by_merchant_from_capital_decision_merchant,114,898,280,Normal
8,8,total_net_count_in_full_28d_by_merchant_from_capital_decision_merchant,10,76,27,Normal
9,9,total_active_day_count_in_full_28d_by_merchant_from_capital_decision_merchant,7,19,11,Normal
